In [1]:
import subprocess
import os
import sys
import shutil
import argparse
import xlrd
from openpyxl import Workbook
import markdown2
from weasyprint import HTML
import stat
import time
import pandas as pd
import soundfile as sf

In [2]:
def transform_file_path(file_path):

    components = file_path.split(os.sep)
    
    preservation_index = components.index('preservation')
    
    components[preservation_index] = r"access\nearline"
    access = r"access\nearline"
    new_file_path = os.sep.join(components[:-1])  # Remove the last directory
    #new_file_path = os.path.join(new_file_path, access)
    print(new_file_path)
    return new_file_path

def conversion_log_modify(name, input, output, format):
    global conversion_log
    file_stat = os.stat(input)
    print(file_stat)
    print(name)
    creation_time = file_stat.st_mtime
    # Convert to a human-readable format
    readable_time = time.ctime(creation_time)
    print(f"The file was created on: {readable_time}")
    print(f"Original Format: {format}")
    current_time_seconds = time.time()
    local_time = time.strftime("%a %b %d %H:%M:%S %Y", time.localtime(current_time_seconds))
    conversion_log = pd.concat([conversion_log, pd.DataFrame([{
                        'File Name': name,
                        'Original Path': input,
                        'New Path': output,
                        'File Conversion': format,
                        'Original File Created': readable_time,
                        'New File Created': local_time,
                    }])], ignore_index=True)

In [3]:
def convert_files(folder_path):

    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} does not exist.")
        return
    global conversion_log
    contents = os.listdir(folder_path)
    
    for content in contents:
        
        content_path = os.path.join(folder_path, content)
        
        if os.path.isdir(content_path):
            print(f"Entering subfolder: {content_path}")
            create = content_path.replace("preservation", "access/nearline")
            os.makedirs(create,mode=0o775, exist_ok=True)
            convert_files(content_path)
        else:
            print("Found file:", content_path)
            if(content.endswith('.mac') or content.endswith('.MAC') or content.endswith('.doc') or content.endswith('.DOC')):
                try: 
                        
                    input_file = content_path
                    name = content.replace('.mac', '')
                    name = name.replace('.MAC', '')
                    name = content.replace('.doc', '')
                    name = name.replace('.DOC', '')
                    output_file_name = f'{name}.pdf'
                    folder_path = os.path.dirname(content_path)
                    output_file = os.path.join(folder_path, output_file_name)
                    output_file = transform_file_path(output_file)
                    output_file_pdf = os.path.join(output_file, output_file_name)
                    print(output_file)
                    convert_mac(input_file, output_file_pdf)
                    conversion_log_modify(content, input_file, output_file_pdf, 'MacWrite to PDF')
            
                except Exception as E:
                    print(E)
                    print("Could not convert!")
            
                

In [4]:
def convert_mac(input_file, output_file):
    try:
        subprocess.run(["python", r"C:\Users\pal10\Desktop\tkapp\unoconv", "-f", "pdf", "-o", output_file, input_file], check=True)
        print("Conversion successful!")
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")

In [5]:
path1 = r"C:\Users\pal10\Desktop\testlotus\preservation"

In [6]:
conversion_log = pd.DataFrame(columns=['File Name', 'Original Path', 'New Path', 'File Conversion', 'Original File Created', 'New File Created'])
path2 = path1.replace('preservation','ObjectDocumentation')
os.makedirs(path2,mode=0o775, exist_ok=True)

In [7]:
convert_files(path1)

Entering subfolder: C:\Users\pal10\Desktop\testlotus\preservation\301121023MWR
Entering subfolder: C:\Users\pal10\Desktop\testlotus\preservation\301121023MWR\OriginalFiles
Found file: C:\Users\pal10\Desktop\testlotus\preservation\301121023MWR\OriginalFiles\action sheet.doc
C:\Users\pal10\Desktop\testlotus\access\nearline\301121023MWR\OriginalFiles
C:\Users\pal10\Desktop\testlotus\access\nearline\301121023MWR\OriginalFiles
Conversion successful!
os.stat_result(st_mode=33206, st_ino=2251799814310412, st_dev=16016024109827288881, st_nlink=1, st_uid=0, st_gid=0, st_size=2144, st_atime=1715785140, st_mtime=1698771609, st_ctime=1715785129)
action sheet.doc
The file was created on: Tue Oct 31 12:00:09 2023
Original Format: MacWrite to PDF
Found file: C:\Users\pal10\Desktop\testlotus\preservation\301121023MWR\OriginalFiles\bump_tt promo.doc
C:\Users\pal10\Desktop\testlotus\access\nearline\301121023MWR\OriginalFiles
C:\Users\pal10\Desktop\testlotus\access\nearline\301121023MWR\OriginalFiles
Con

In [ ]:
print(conversion_log)